# Serial Monitor Data Collector

This python scripts below will monitor the given serial port. From that given port it will continously collects the seen data and then saves it in a usable .csv file for us. This is how we collect the necessary sign data from the gloves for our model's training data.

### COM port scanning

This part executes a routine port scan that will print out the available COM ports to chose from.

In [7]:
import sys
import glob
import serial

def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

print(serial_ports())

['COM3']


### Connecting to the COM port & .csv file creation

Below, the serial connection will be established for the given parameters. Also the .csv file will be created initially.

In [11]:
import serial
import csv

esp32_port = "COM3" # The used COM port of the ESP32.
baud_rate = 112500 # The ESP32 runs at 112500 baud rate (can be changed if necessary).
file_name="collected_data.csv" # The name of the .csv file to be generated.

ser = serial.Serial(esp32_port, baud_rate)
print("Connected to ESP32 on port:" + esp32_port)
file = open(file_name, "a")
print("Created file")

Connected to ESP32 on port:COM3
Created file


### Reading serial COM port

Continously monitoring the serial connection on the given port and while the connection exists and data is being retrieved from it to a local collection.

In [14]:
sensor_data = [] #store data

# Collect the samples.
while True:
    raw_data = ser.readline()
    data_string = raw_data.decode('utf-8')
    data = data_string[0:][:-2]
    print(data)
    readings = data.split(",")
    if "0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0," not in data and len(readings) == 32:
        readings.pop(31)
        sensor_data.append(readings)

-172.502182,0,0,0,0,0,0,-6.385616,-4.914854,-6.803174,12.077169,-99.857162,92.337517,-7.989059,-7.141891,-6.699616,10.194849,-29.310089,-67.221329,0,0,0,0,0,0,
fel,2.926488,-0.441078,-1.713436,-73.192886,66.343773,188.981644,0,0,0,0,0,0,-8.389856,-6.605611,-5.091181,26.609024,77.019638,-43.923367,-7.240521,-6.893349,-5.138318,-35.824715,48.596447,40.759476,8.411429,6.196108,5.790317,28.070290,44.473473,11.749684,
fel,0.216203,-1.292454,-2.366971,65.628464,65.616844,138.564056,0,0,0,0,0,0,-5.054375,-3.905571,-0.210442,-15.551215,23.429852,-30.793369,-4.231289,-2.830376,0.167808,25.573467,11.763483,-45.573078,0,0,0,0,0,0,
fel,-0.958036,-1.440868,0.126096,65.600006,140.351822,-80.716515,0,0,0,0,0,0,-0.628002,0.136540,2.514880,-14.975506,26.248692,-92.236176,0,0,0,0,0,0,0,0,0,0,0,0,
fel,-1.316570,-0.630460,2.380210,-20.832790,57.284107,-26.551594,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.360665,3.440024,5.057559,-14.727086,-26.194059,5.754138,
fel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

KeyboardInterrupt: 

### Saving data to the .csv file

The collected data is being written to the created .csv file, hence completing the data collection from the serial port.

In [15]:
with open(file_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(sensor_data)

print("Data collection complete!")
file.close()

Data collection complete!
